In [3]:
import wave
import struct
import os
import numpy as np
import json
from scipy.fft import fft as scipy_fft

# === WAV LOADER ===
def read_wav(filename):
    with wave.open(filename, 'rb') as wav_file:
        n_channels, sampwidth, framerate, n_frames, _, _ = wav_file.getparams()
        raw_data = wav_file.readframes(n_frames)
        fmt = "<" + "h" * (n_frames * n_channels)
        data = struct.unpack(fmt, raw_data)
        print(f"📂 Membaca file: {filename} (sample rate: {framerate} Hz, total frames: {n_frames})")
        return np.array(data), framerate

# === DOMAIN WAKTU ===
def zero_crossing_rate(samples):
    # Vectorized zero crossing rate calculation
    signs = np.sign(samples)
    sign_changes = np.diff(signs)
    zcr = np.count_nonzero(sign_changes) / len(samples)
    print(f"🔹 Zero Crossing Rate: {zcr:.5f}")
    return zcr

def energy(samples):
    # Vectorized energy calculation
    en = np.mean(samples**2)
    print(f"🔹 Energi sinyal: {en:.2f}")
    return en

# === DOMAIN FREKUENSI (FFT + Freq Dominan) ===
def fft(samples):
    print(f"🔸 Melakukan FFT pada {len(samples)} sampel...")
    # Use NumPy's FFT implementation
    fft_result = np.fft.fft(samples)
    magnitudes = np.abs(fft_result)
    return magnitudes

def dominant_freq(magnitudes, sample_rate):
    # Find dominant frequency using NumPy
    max_index = np.argmax(magnitudes)
    freq = max_index * sample_rate / len(magnitudes)
    print(f"🔹 Frekuensi dominan: {freq:.2f} Hz")
    return freq

# === MFCC ===
def pre_emphasis(signal, coeff=0.97):
    # Vectorized pre-emphasis filter
    emphasized = np.zeros_like(signal)
    emphasized[0] = signal[0]
    emphasized[1:] = signal[1:] - coeff * signal[:-1]
    return emphasized

def hamming_window(N):
    # Use NumPy's hamming window
    return np.hamming(N)

def hz_to_mel(hz):
    return 2595 * np.log10(1 + hz / 700)

def mel_to_hz(mel):
    return 700 * (10**(mel / 2595) - 1)

def mel_filterbank(n_filters, N_fft, sample_rate):
    # Vectorized mel filterbank creation
    low_mel = hz_to_mel(0)
    high_mel = hz_to_mel(sample_rate / 2)
    
    # Create mel points
    mel_points = np.linspace(low_mel, high_mel, n_filters + 2)
    hz_points = mel_to_hz(mel_points)
    bin_points = np.floor(hz_points * N_fft / sample_rate).astype(int)
    
    # Create filter bank matrix
    filters = np.zeros((n_filters, N_fft // 2))
    
    for i in range(1, n_filters + 1):
        left = bin_points[i - 1]
        center = bin_points[i]
        right = bin_points[i + 1]
        
        # Left slope
        for j in range(left, center):
            if j < filters.shape[1]:
                filters[i - 1, j] = (j - left) / (center - left)
        
        # Right slope
        for j in range(center, right):
            if j < filters.shape[1]:
                filters[i - 1, j] = (right - j) / (right - center)
    
    return filters

def apply_filterbanks(magnitudes, filters):
    # Vectorized filterbank application
    # Ensure magnitudes length matches filter bank width
    mag_len = min(len(magnitudes), filters.shape[1])
    magnitudes_truncated = magnitudes[:mag_len]
    filters_truncated = filters[:, :mag_len]
    
    # Apply filters using matrix multiplication
    energies = np.dot(filters_truncated, magnitudes_truncated)
    # Add small epsilon to avoid log(0)
    energies = np.log(energies + 1e-10)
    return energies

def dct(signal):
    # Vectorized DCT implementation
    N = len(signal)
    n = np.arange(N)
    k = np.arange(N).reshape(-1, 1)
    
    # DCT matrix
    dct_matrix = np.cos(np.pi * k * (2 * n + 1) / (2 * N))
    result = np.dot(dct_matrix, signal)
    return result

def mfcc(samples, sample_rate, num_filters=26, num_coeffs=13):
    print("🔸 Menghitung MFCC...")
    
    # Pre-emphasis
    emphasized = pre_emphasis(samples)
    
    # Windowing
    frame_size = int(0.025 * sample_rate)
    frame = emphasized[:frame_size]
    hamming = hamming_window(len(frame))
    windowed = frame * hamming
    
    # FFT
    spectrum = fft(windowed)
    
    # Mel filterbank
    filters = mel_filterbank(num_filters, len(spectrum) * 2, sample_rate)
    
    # Apply filterbanks
    energies = apply_filterbanks(spectrum, filters)
    
    # DCT
    cepstrals = dct(energies)
    mfccs = cepstrals[:num_coeffs]
    
    print(f"🔹 MFCC (13 coeff): {np.round(mfccs, 2).tolist()}")
    return mfccs.tolist()

# === EKSTRAKSI FITUR ===
def extract_features(file_path):
    print(f"\n📥 Ekstraksi fitur dari: {file_path}")
    samples, sample_rate = read_wav(file_path)
    
    zcr = zero_crossing_rate(samples)
    en = energy(samples)
    
    # Use first 512 samples for FFT
    fft_samples = samples[:512] if len(samples) >= 512 else samples
    fft_mags = fft(fft_samples)
    dom_freq = dominant_freq(fft_mags, sample_rate)
    
    mfcc_feats = mfcc(samples, sample_rate)
    
    return [zcr, en, dom_freq] + mfcc_feats

# === LOAD DATASET ===
def load_dataset(folder_path):
    data = []
    labels = []
    print(f"📁 Memuat dataset dari: {folder_path}")
    
    for label in os.listdir(folder_path):
        label_folder = os.path.join(folder_path, label)
        if not os.path.isdir(label_folder):
            continue
            
        print(f"\n🗂️ Label: {label}")
        for filename in os.listdir(label_folder):
            if not filename.lower().endswith('.wav'):
                continue  # Lewati .png dan .py
                
            path = os.path.join(label_folder, filename)
            try:
                features = extract_features(path)
                data.append(features)
                labels.append(label)
            except Exception as e:
                print(f"⚠️ Gagal memproses {path}: {e}")
    
    return np.array(data), labels

def encode_labels(labels):
    unique_labels = list(set(labels))
    label_to_int = {label: i for i, label in enumerate(unique_labels)}
    print(f"\n🔢 Label encoding: {label_to_int}")
    encoded_labels = np.array([label_to_int[label] for label in labels])
    return encoded_labels, label_to_int

def train_test_split(data, labels, ratio=0.8, shuffle=False):
    n_train = int(len(data) * ratio)
    
    if shuffle:
        # Shuffle data before splitting
        indices = np.random.permutation(len(data))
        train_indices = indices[:n_train]
        test_indices = indices[n_train:]
        print(f"🔄 Split data (shuffled): {n_train} train, {len(data) - n_train} test")
        return (data[train_indices], labels[train_indices],
                data[test_indices], labels[test_indices])
    else:
        # Sequential split (like original)
        print(f"🔄 Split data (sequential): {n_train} train, {len(data) - n_train} test")
        return (data[:n_train], labels[:n_train],
                data[n_train:], labels[n_train:])

# === K-NN ===
def euclidean_distance(a, b):
    # Vectorized Euclidean distance
    return np.sqrt(np.sum((a - b)**2))

def predict_knn(train_data, train_labels, test_point, k=3):
    # Vectorized distance calculation for all training points
    distances = np.array([euclidean_distance(test_point, td) for td in train_data])
    
    # Get k nearest neighbors
    k_indices = np.argsort(distances)[:k]
    k_labels = train_labels[k_indices]
    
    # Return most common label
    unique_labels, counts = np.unique(k_labels, return_counts=True)
    return unique_labels[np.argmax(counts)]

def evaluate(train_data, train_labels, test_data, test_labels):
    print("\n🧪 Evaluasi model (k-NN)...")
    
    predictions = []
    for i, test_point in enumerate(test_data):
        pred = predict_knn(train_data, train_labels, test_point)
        predictions.append(pred)
        actual = test_labels[i]
        print(f"   ▸ Test {i+1}: Pred = {pred}, Label = {actual} --> {'✅' if pred == actual else '❌'}")
    
    predictions = np.array(predictions)
    accuracy = np.mean(predictions == test_labels)
    print(f"\n🎯 Akurasi: {accuracy*100:.2f}%")
    return accuracy

# === SIMPAN MODEL ===
def save_model(train_data, train_labels, label_map, filename='siren_model.json'):
    model = {
        'data': train_data.tolist(),  # Convert numpy array to list for JSON serialization
        'labels': train_labels.tolist(),
        'label_map': label_map
    }
    
    with open(filename, 'w') as f:
        json.dump(model, f)
    print(f"\n✅ Model disimpan sebagai: {filename}")

# === MAIN ===
if __name__ == "__main__":
    # Set random seed for reproducibility
    np.random.seed(42)
    
    dataset_path = "sounds"  # Ganti sesuai lokasi foldermu

    data, labels = load_dataset(dataset_path)
    labels_encoded, label_map = encode_labels(labels)

    train_data, train_labels, test_data, test_labels = train_test_split(data, labels_encoded, shuffle=False)

    evaluate(train_data, train_labels, test_data, test_labels)

    save_model(train_data, train_labels, label_map)

📁 Memuat dataset dari: sounds

🗂️ Label: ambulance

📥 Ekstraksi fitur dari: sounds\ambulance\sound_1.wav
📂 Membaca file: sounds\ambulance\sound_1.wav (sample rate: 44100 Hz, total frames: 132300)
🔹 Zero Crossing Rate: 0.34868
🔹 Energi sinyal: 537020.02
🔸 Melakukan FFT pada 512 sampel...
🔹 Frekuensi dominan: 0.00 Hz
🔸 Menghitung MFCC...
🔸 Melakukan FFT pada 1102 sampel...
🔹 MFCC (13 coeff): [-598.67, -0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0]

📥 Ekstraksi fitur dari: sounds\ambulance\sound_10.wav
📂 Membaca file: sounds\ambulance\sound_10.wav (sample rate: 44100 Hz, total frames: 132300)
🔹 Zero Crossing Rate: 0.35828
🔹 Energi sinyal: 524630.68
🔸 Melakukan FFT pada 512 sampel...
🔹 Frekuensi dominan: 0.00 Hz
🔸 Menghitung MFCC...
🔸 Melakukan FFT pada 1102 sampel...
🔹 MFCC (13 coeff): [144.18, -19.75, 3.29, -0.23, -2.02, 3.63, -3.64, 0.64, 1.13, 0.17, 1.84, -0.34, 1.69]

📥 Ekstraksi fitur dari: sounds\ambulance\sound_100.wav
📂 Membaca file: sounds\ambulance\sound_100.